In [90]:
import gpxpy
import gpxpy.gpx
from glob import glob
import os
import datetime
import pandas as pd
from pandas import DataFrame
import matplotlib.pyplot as plt
import numpy as np
import seawater as sw
import mplleaflet

In [91]:
def load_run_data(gpx_path, filter=""):
    gpx_files = glob(os.path.join(gpx_path, filter + "*.gpx"))
    run_data = []
    counter = 0
    for file_idx, gpx_file in enumerate(gpx_files): 
        gpx = gpxpy.parse(open(gpx_file, 'r'))
        counter = counter + 1
        # Loop through tracks
        for track_idx, track in enumerate(gpx.tracks):
            track_name = track.name
            track_time = track.get_time_bounds().start_time
            track_length = track.length_3d()
            track_duration = track.get_duration()
            track_speed = track.get_moving_data().max_speed
            
            for seg_idx, segment in enumerate(track.segments):
                segment_length = segment.length_3d()
                for point_idx, point in enumerate(segment.points):
                    run_data.append([counter, os.path.basename(gpx_file), track_idx, track_name, 
                                     track_time, track_length, track_duration, track_speed, 
                                     seg_idx, segment_length, point.time, point.latitude, 
                                     point.longitude, point.elevation, segment.get_speed(point_idx)])
    return run_data

In [92]:
rundata = load_run_data(gpx_path='./dat/run/', filter="")
ridedata = load_run_data(gpx_path='./dat/ride/', filter="")

df_run = DataFrame(rundata, columns=['File_Index', 'File_Name', 'Index', 'Name',
                              'Time', 'Length', 'Duration', 'Max_Speed',
                              'Segment_Index', 'Segment_Length', 'Point_Time', 'Point_Latitude',
                              'Point_Longitude', 'Point_Elevation', 'Point_Speed'])

df_ride = DataFrame(ridedata, columns=['File_Index', 'File_Name', 'Index', 'Name',
                              'Time', 'Length', 'Duration', 'Max_Speed',
                              'Segment_Index', 'Segment_Length', 'Point_Time', 'Point_Latitude',
                              'Point_Longitude', 'Point_Elevation', 'Point_Speed'])

cols = ['File_Index', 'Time', 'Length', 'Duration', 'Max_Speed']

tracks_run = df_run[cols].copy()
tracks_run['Length'] /= 1e3
tracks_run.drop_duplicates(inplace=True)

tracks_ride = df_ride[cols].copy()
tracks_ride['Length'] /= 1e3
tracks_ride.drop_duplicates(inplace=True)

In [93]:
# Find maximum value for the DataFrame and create an enumerated list of numbers 1 -> max, looping through this
# to individually lay down routes.
numSegs = df_run['File_Index'].max()
seg_count_run = np.arange(1,numSegs + 1) # Second number is not inclusive

numSegs = df_ride['File_Index'].max()
seg_count_ride = np.arange(1,numSegs + 1) 

In [94]:
# Only important if we want to make a quiver plot with vectors!

#_, angles_run = sw.dist(df_run['Point_Latitude'], df_run['Point_Longitude'])
#angles_run = np.r_[0, np.deg2rad(angles_run)]

#_, angles_ride = sw.dist(df_ride['Point_Latitude'], df_ride['Point_Longitude'])
#angles_ride = np.r_[0, np.deg2rad(angles_ride)]

#r = df_run['Point_Speed'] / df_run['Point_Speed'].max()
#kw = dict(window_len=31, window='hanning')
#df_run['u'] = r * np.cos(angles_run)
#df_run['v'] = r * np.sin(angles_run)

#r = df_ride['Point_Speed'] / df_ride['Point_Speed'].max()
#kw = dict(window_len=31, window='hanning')
#df_ride['u'] = r * np.cos(angles_ride)
#df_ride['v'] = r * np.sin(angles_ride)

In [96]:
fig, ax = plt.subplots()
df_run = df_run.dropna() # Just drop columns with NaN... usually only the first.
df_ride = df_ride.dropna()

for idx in seg_count_run:
    locs = df_run[df_run['File_Index'] == idx].index.tolist()
    ax.plot(df_run['Point_Longitude'][locs], df_run['Point_Latitude'][locs],
            color='#000080', linewidth=2, alpha=0.75)

for idx in seg_count_ride:
    locs = df_ride[df_ride['File_Index'] == idx].index.tolist()
    ax.plot(df_ride['Point_Longitude'][locs], df_ride['Point_Latitude'][locs],
            color='#C40000', linewidth=2, alpha=0.75)

mplleaflet.show(fig=fig, tiles='stamen_toner') 



#df_ride = df_ride.dropna()

#ax.plot(df_run['Point_Longitude'], df_run['Point_Latitude'],
#        color='#000080', linewidth=2, alpha=0.75)

#ax.plot(df_ride['Point_Longitude'], df_ride['Point_Latitude'],
#        color='#C40000', linewidth=2, alpha=0.75)

#sub = 10
#ax.quiver(df['Point_Longitude'][::sub], df['Point_Latitude'][::sub],
#          df['u'][::sub], df['v'][::sub], color='black',
#          alpha=0.75, scale=None, headwidth=4)

#mplleaflet.show(fig=fig, tiles='stamen_toner') 